## Explore dataset

In [ ]:
import os

Load the metadata file into a pandas DataFrame.

In [ ]:
import numpy as np
import pandas as pd

products_metadata = pd.read_pickle('/shared_dir/notebooks/files_metadata.pickle')
products_metadata[0]

From metadata extract all regions patches and store in longitude/latitude dictionary.

In [ ]:
def compute_aprox_center(coords):
    ys = sorted(coords[:,1])
    xs = sorted(coords[:,0])
    center_approx = ((xs[3]+xs[1])/2,(ys[3]+ys[1])/2)
    return center_approx

In [ ]:
products_id = [md["productId"] for md in products_metadata]
products_geom = [md["geometry"]["geometries"][0] for md in products_metadata]
products_geom_coordinates = [md["coordinates"][0] for md in products_geom]
products_geom_centers = [compute_aprox_center(np.array(md["coordinates"][0][1:])) for md in products_geom ]

coordinates = {"lon": [], "lat": []}
for el in products_geom_coordinates:
    for item in el:
        coordinates["lon"].append(item[0])
        coordinates["lat"].append(item[1])

Load geojson regions that we know contains areas with high crater density and overlay the dataset images patches. Use GeoPandas with inbuilt projection.

In [ ]:
! ls /shared_dir

In [ ]:
import geopandas as gpd
from os import listdir
from os.path import join
%matplotlib notebook
geo_regions = listdir('/shared_dir/data/geometries/')

for geo in geo_regions[0:1]:
    df = gpd.read_file(join('/shared_dir/data/geometries/', geo))
    print(df.columns)
    print(df["geometry"])

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(10, 5))

ax = plt.subplot()
ax.grid(True)

ax.set_title("2D projection")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

ax.set_ylim([10, 30])
ax.set_xlim([65, 80])

# blue
for geo in geo_regions:
    df = gpd.read_file(join('/shared_dir/data/geometries/', geo))
    df.boundary.plot(ax=ax, facecolor=None, linewidth=1)
        
# purple
for el in products_geom_coordinates:
    ax.add_patch(patches.Polygon(el, closed=True, facecolor=None, edgecolor="orange", fill=False, alpha=0.7))

for cent in products_geom_centers:
    ax.add_patch(patches.Circle(cent, 0.05, color="red"))
    
fig.savefig("intersections.png", dpi=300)

Create GeoDataFrames from preselected geometries containing craters and dataset geometries. This should help filtering the dataset via intersection or overlap in order to filter out areas with small craters density.

In [ ]:
from shapely.geometry import Polygon

p1_list, p1_id = [], []
for geo in geo_regions:
    df = gpd.read_file(join('/shared_dir/data/geometries/', geo))
    p1_list.append([item for item in df.geometry][0])
    p1_id.append(df.name[0])
    
p1 = gpd.GeoSeries(p1_list, crs="EPSG:4326")
rois = gpd.GeoDataFrame({'geometry': p1, 'id': p1_id})

p2_list = []
for el in products_geom_coordinates:
    p2_list.append([Polygon([(item[0], item[1]) for item in el])][0])
    
p2 = gpd.GeoSeries(p2_list, crs="EPSG:4326")
imgs = gpd.GeoDataFrame({'geometry': p2, 'id': products_id})

rois.head(), imgs.head()

Intersection allows to define the areas of geometry which are contained by both sets. The issue is that it works row by row, one geometry against the other. Given that we have 20 region of interest against 905 images, the intersection ends up empy. We should find a solution that performs the intersection in each permutation of the two dataframes.

In [ ]:
intersected = rois.intersection(imgs, align=True)     
intersected[~intersected.is_empty][~intersected.isna()]

The same as for the intersection can be said about the <code>contains</code> method.

In [ ]:
contained = rois.contains(imgs, align=True)     
contained[~contained == False]

Let's try to iterate the intersection geometry-wise and save the intersection in a new GeoDataFrame with corresponding geometry IDs and centers.

In [ ]:
intersected_list, intersected_ids = [], []
for geo in geo_regions:
    df = gpd.read_file(join('/shared_dir/data/geometries/', geo))
    roi = gpd.GeoSeries([item for item in df.geometry][0], crs="EPSG:4326")

    for idx, el in enumerate(products_geom_coordinates):
        img = gpd.GeoSeries([Polygon([(item[0], item[1]) for item in el])][0], crs="EPSG:4326")

        intersected = roi.intersection(img)
        if not intersected.is_empty.any():
            intersected_list.append(intersected[0])
            intersected_ids.append(products_id[idx])

intersected_series = gpd.GeoSeries(intersected_list, crs="EPSG:4326")
intersected_df = gpd.GeoDataFrame({'geometry': intersected_series, 
                                   'id': intersected_ids,
                                   'center': intersected_series.centroid,
                                   'area': intersected_series.area,
                                   'length': intersected_series.length,
                                   'sqeql': np.sqrt(intersected_series.area)
                                  })
            
intersected_df.head()

Plot the intersections between geometries and dataset.

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.subplot()
ax.grid(True)

ax.set_title("2D projection")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_ylim([-90, 90])
ax.set_xlim([-180, 180])

# blue
for geo in geo_regions:
    df = gpd.read_file(join('/shared_dir/data/geometries/', geo))
    df.boundary.plot(ax=ax, fc=None)
        
# purple
intersected_df.boundary.plot(ax=ax, fc=None, ec='g', lw=0.2)

Length of intersected dataset:

In [ ]:
len(intersected_df)

Find average square equivalent length for the intersected dataset.

In [ ]:
intersected_df['sqeql'].mean()

Now let's order the geometries in the GeoDataFrame by their global location. We do so by computing the x- and y-coordinates of the centroids of each geometry. To do so we need to reduce from 2d to 1d, thus we average the x- and y-coordinates and then sort the geometries by this averaged value.

In [ ]:
sorted_intersected_df = (intersected_df
                         .assign(x=lambda df: df['center'].x)
                         .assign(y=lambda df: df['center'].y)
                         .assign(aval=lambda df: df[['x', 'y']].mean(axis=1))
                         .sort_values(by=['aval']))

len(sorted_intersected_df)

In [ ]:
sorted_intersected_df.head()

Indexes are now not ordered as well as not contiguous. Let's reindex all the surviving rows.

In [ ]:
sorted_intersected_df = sorted_intersected_df.reset_index(drop=True)
sorted_intersected_df.head()

In order to avoid excessive overlap in the dataset, we filter out images that are consider not distinct. In this algorithm, we define two distinct images as having the centroid of their geometry that has vectorial distance greater than half of the mean square equivalent length of the dataset itself. If two images are found to be not distinct we drop keep the first (lower located) image and compare to the following. This way, having an ordered dataset, we can maximise the number of distinct images comprised in a cluster.

In [ ]:
meanl = intersected_df['sqeql'].mean()
filtered_intersected_df = sorted_intersected_df

idx = 0
while idx < len(filtered_intersected_df)-1:
    current = filtered_intersected_df.loc[idx,:]['geometry']
    nextone = filtered_intersected_df.loc[idx+1,:]['geometry']
    centers_distance = current.distance(nextone)
    if centers_distance >= meanl/4:
        idx += 1        
    else:
        filtered_intersected_df = filtered_intersected_df.drop(idx+1).reset_index(drop=True)
        
len(sorted_intersected_df), len(filtered_intersected_df)       

In [ ]:
filtered_intersected_df['id']